In [21]:
import os

In [16]:
def get_incomplete_files(dir_path, dst_path):

    # isolate files that are not present in dst_path but are there in dir_path, the filename is the same, but the extension is different
    img_files = [file for file in os.listdir(
        dir_path) if file.endswith(".png") or file.endswith(".jpg")]
    img_summaries = [file for file in os.listdir(
        dst_path) if file.endswith(".txt")]

    # compare using file extensions
    incomplete_files = [
        file for file in img_files if f"{file}.txt" not in img_summaries]
    return incomplete_files

In [ ]:
images = "../data/temporary_data/images"
image_summaries = "../data/temporary_data/image_summaries"

#get the number of files in both directories and print side-by-side
images_files = os.listdir(images)
image_summaries_files = os.listdir(image_summaries)

print("Images: ", len(images_files), "Image Summaries: ", len(image_summaries_files))

Images:  82 Image Summaries:  76


Images:  82 Image Summaries:  82


In [18]:
incomplete_files = get_incomplete_files(images, image_summaries)

In [19]:
incomplete_files

['SFTIN_WMS_Account-UserGuide_page_1_image_4.png',
 'SFTIN_WMS_Account-UserGuide_page_4_image_2.png',
 'SFTIN_WMS_Account-UserGuide_page_1_image_3.png',
 'SFTIN_WMS_Account-UserGuide_page_1_image_1.png',
 'SFTIN_WMS_Account-UserGuide_page_7_image_2.png',
 'SFTIN_WMS_Account-UserGuide_page_6_image_2.png']

## Token Analysis

In [24]:
import tiktoken

def num_tokens_from_string(string: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [38]:
data_dir = "../data/temporary_data/text"
texts = {}
for file in os.listdir(data_dir):
    with open(os.path.join(data_dir, file), "r") as f:
        texts[file] = f.read()
    

In [45]:
token_sizes = [num_tokens_from_string(text[1]) for text in texts.items()]

In [47]:
token_sizes

[782, 481, 544, 606, 1060, 619, 437, 1322, 582, 965, 4092]

## Tagging

In [58]:
from langchain_community.chat_message_histories.file import FileChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
import io
import re
import tiktoken
from IPython.display import HTML, display
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from PIL import Image
import pickle
from langchain_core.documents import Document
import uuid
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, AIMessagePromptTemplate, StringPromptTemplate
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnableWithMessageHistory, RunnableLambda
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_core.runnables import RunnableWithMessageHistory, ConfigurableFieldSpec
import base64
import json
load_dotenv()

True

In [49]:
tagging_prompt = """
###Objective###
- Based on the text_content provide and appropriate tag for the text.

###Rules###
- Select as many tags as you think are appropriate for the text.

###Tags###
**WMS**
   - **Description**: Refers to documents related to the Warehouse Management System (WMS), including setup, configuration, and operational guidance specific to warehouse management processes.

**Administrative**
   - **Description**: Covers documents focused on administrative tasks such as system configuration, user management, account management, and other setup-related activities within the WMS.

**User Interface**
   - **Description**: Includes guides and documentation related to the customization, configuration, and design of the user interface within the WMS, focusing on UI objects, templates, and screen layouts.

**Operational Processes**
   - **Description**: Pertains to documents that outline various operational tasks within the WMS, such as batch management, inbound and outbound logistics, and daily warehouse activities.

**Automation**
   - **Description**: Refers to guides detailing automated processes within the WMS, including features that enhance efficiency by reducing manual intervention, such as auto-loading trucks and automated inbound inspections.

**Inbound**
   - **Description**: Focuses on documentation related to inbound logistics processes, including the receiving, inspection, and handling of goods as they enter the warehouse.

**Outbound**
   - **Description**: Covers documents related to outbound logistics processes, such as picking, packing, and shipping goods from the warehouse, as well as related operational tasks.

**Integration**
   - **Description**: Documentation that deals with the integration of the WMS with other systems or processes, ensuring seamless operation and data flow across different platforms.

**Compliance**
   - **Description**: Documentation that addresses regulatory or compliance-related requirements within the logistics and warehouse management processes, ensuring adherence to industry standards.### **WMS**
   - **Description**: Refers to documents related to the Warehouse Management System (WMS), including setup, configuration, and operational guidance specific to warehouse management processes.

**Administrative**
   - **Description**: Covers documents focused on administrative tasks such as system configuration, user management, account management, and other setup-related activities within the WMS.

**User Interface**
   - **Description**: Includes guides and documentation related to the customization, configuration, and design of the user interface within the WMS, focusing on UI objects, templates, and screen layouts.

**Operational Processes**
   - **Description**: Pertains to documents that outline various operational tasks within the WMS, such as batch management, inbound and outbound logistics, and daily warehouse activities.

**Automation**
   - **Description**: Refers to guides detailing automated processes within the WMS, including features that enhance efficiency by reducing manual intervention, such as auto-loading trucks and automated inbound inspections.

**Inbound**
   - **Description**: Focuses on documentation related to inbound logistics processes, including the receiving, inspection, and handling of goods as they enter the warehouse.

**Outbound**
   - **Description**: Covers documents related to outbound logistics processes, such as picking, packing, and shipping goods from the warehouse, as well as related operational tasks.

**Integration**
   - **Description**: Documentation that deals with the integration of the WMS with other systems or processes, ensuring seamless operation and data flow across different platforms.

**Compliance**
   - **Description**: Documentation that addresses regulatory or compliance-related requirements within the logistics and warehouse management processes, ensuring adherence to industry standards.

`text_content`: {text_content}
"""

tagging_prompt = ChatPromptTemplate.from_template(tagging_prompt)

In [50]:
class TaggingSchema(BaseModel):
    explanation: str = Field(..., title="Explanation", description="Detailed reasoning for the possible tags that fit the text content")
    tags: list[str] = Field(..., title="Tags", description="Tags that are appropriate for the text content")

In [51]:
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [52]:
structured_llm = llm.with_structured_output(TaggingSchema)

In [53]:
tagging_chain = tagging_prompt | structured_llm

In [54]:
processed_texts = {}

for file, text in texts.items():
    print(f"Processing {file}")
    outputs = tagging_chain.invoke({"text_content": text})
    #save a dictionary of the text and the tags for each file
    processed_texts[file] = outputs


Processing SFTIN_WMS_Address-UserGuide.txt
Processing SFTIN_WMS_AdhocUIDefinition-UserGuide.txt
Processing SFTIN_WMS_AutoLoadTruckBySPN-UserGuide.txt
Processing SFTIN_WMS_AdhocQueryParamSetup-UserGuide.txt
Processing SFTIN_WMS_AutoHoldInboundInspection-UserGuide.txt
Processing SFTIN_WMS_AdhocUIObject-UserGuide.txt
Processing SFTIN_WMS_BOLType-UserGuide.txt
Processing SFTIN_WMS_Account-UserGuide.txt
Processing SFTIN_WMS_BatchPreference-UserGuide.txt
Processing SFTIN_WMS_AdhocMasterTemplate-UserGuide.txt
Processing SFTN_WMS_SecurityProfile-UserGuide.txt


In [56]:
json_output = {}
for file, output in processed_texts.items():
    json_output[file] = output.dict()

In [59]:
output_dir = "../data/temporary_data/file_metadata"

os.makedirs(output_dir, exist_ok=True)
#save json file
with open(os.path.join(output_dir, "sample_metadata.json"), "w") as f:
    json.dump(json_output, f)